In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 13.5 MB/s eta 0:00:00


In [5]:
!pip install ekphrasis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.8/83.8 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 5.6 MB/s eta 0:00:00


In [6]:
pip install plotly==4.5.4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 15.6 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.24.1
    Uninstalling plotly-5.24.1:
      Successfully uninstalled plotly-5.24.1


In [7]:
!pip install transformers==4.2.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.2/184.2 kB 9.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 39.0 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tokenizers
Failed to build tokenizers
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (tokenizers)


In [8]:
import numpy as np
import pandas as pd
import string
from nltk.corpus import stopwords
import re
import os

from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons

In [9]:
text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['url', 'email', 'percent', 'money', 'phone', 'user',
        'time', 'url', 'date', 'number'],
    # terms that will be annotated
    annotate={"hashtag", "allcaps", "elongated", "repeated",
        'emphasis', 'censored'},
    fix_html=True,  # fix HTML tokens

    # corpus from which the word statistics are going to be used
    # for word segmentation
    segmenter="twitter",

    # corpus from which the word statistics are going to be used
    # for spell correction
    corrector="twitter",

    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=True,  # spell correction for elongated words

    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,

    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
)

/usr/local/lib/python3.11/dist-packages/ekphrasis/classes/tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


Word statistics files not found!
Downloading... done!
Unpacking... done!
Reading twitter - 1grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_1grams.txt
Reading twitter - 2grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_2grams.txt
Reading twitter - 1grams ...


/usr/local/lib/python3.11/dist-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


In [10]:
def print_text(texts,i,j):
    for u in range(i,j):
        print(texts[u])
        print()

In [11]:
df = pd.read_csv('/content/drive/My Drive/offenseval/olid-training-v1.0.tsv',delimiter='\t',encoding='utf-8')
print(list(df.columns.values)) #file header
print(df.head(5)) #last N rows

['id', 'tweet', 'subtask_a', 'subtask_b', 'subtask_c']
      id                                              tweet subtask_a  \
0  86426  @USER She should ask a few native Americans wh...       OFF   
1  90194  @USER @USER Go home you’re drunk!!! @USER #MAG...       OFF   
2  16820  Amazon is investigating Chinese employees who ...       NOT   
3  62688  @USER Someone should'veTaken" this piece of sh...       OFF   
4  43605  @USER @USER Obama wanted liberals &amp; illega...       NOT   

  subtask_b subtask_c  
0       UNT       NaN  
1       TIN       IND  
2       NaN       NaN  
3       UNT       NaN  
4       NaN       NaN  


In [12]:
df.replace(np.nan, 'NA', inplace=True)

In [13]:
df.head(5)

,id,tweet,subtask_a,subtask_b,subtask_c
0,86426,@USER She should ask a few native Americans wh...,OFF,UNT,NA
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND
2,16820,Amazon is investigating Chinese employees who ...,NOT,NA,NA
3,62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NA
4,43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NA,NA


In [14]:
text_array = df["tweet"]
labels = df["subtask_a"]
labels_target = df["subtask_b"]
print(len(text_array))
print_text(text_array,0,10)

13240
@USER She should ask a few native Americans what their take on this is.

@USER @USER Go home you’re drunk!!! @USER #MAGA #Trump2020 👊🇺🇸👊 URL

Amazon is investigating Chinese employees who are selling internal data to third-party sellers looking for an edge in the competitive marketplace. URL #Amazon #MAGA #KAG #CHINA #TCOT

@USER Someone should'veTaken" this piece of shit to a volcano. 😂"

@USER @USER Obama wanted liberals &amp; illegals to move into red states

@USER Liberals are all Kookoo !!!

@USER @USER Oh noes! Tough shit.

@USER was literally just talking about this lol all mass shootings like that have been set ups. it’s propaganda used to divide us on major issues like gun control and terrorism

@USER Buy more icecream!!!

@USER Canada doesn’t need another CUCK! We already have enough #LooneyLeft #Liberals f**king up our great country! #Qproofs #TrudeauMustGo



In [15]:
original = text_array

In [16]:
from collections import Counter

In [17]:
df_test_labels_b = pd.read_csv('/content/drive/My Drive/offenseval/labels-levelb.csv', header=None)
print(len(df_test_labels_b))
lol = df_test_labels_b[1]
print(Counter(lol))
df_test_labels_b.head(5)

240
Counter({'TIN': 213, 'UNT': 27})


,0,1
0,15923,TIN
1,60133,TIN
2,83681,TIN
3,65507,TIN
4,12588,UNT


In [18]:
labels_target_test = []

In [19]:
df_test_text = pd.read_csv('/content/drive/My Drive/offenseval/testset-levela.tsv',delimiter='\t',encoding='utf-8')
print(list(df_test_text.columns.values)) #file header
print(df_test_text.head(5)) #first N rows

df_test_labels = pd.read_csv('/content/drive/My Drive/offenseval/labels-levela.csv', header=None)
print(list(df_test_labels.columns.values))
print(df_test_labels.head(5))

count = 0
j = 0
for i in range(0,len(df_test_text["id"])):
    if df_test_labels[1][i] == "OFF":
        if df_test_labels[0][i] == df_test_labels_b[0][j]:
            labels_target_test.append(df_test_labels_b[1][j])
            j = j + 1
    else:
        labels_target_test.append("NA")

print(len(df_test_text["id"]))
print(count)

text_array_test = df_test_text["tweet"]
labels_test = df_test_labels[1]
print("Checking length of validation set")
print(len(text_array_test),len(labels_test))

['id', 'tweet']
      id                                              tweet
0  15923  #WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...
1  27014  #ConstitutionDay is revered by Conservatives, ...
2  30530  #FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendmen...
3  13876  #Watching #Boomer getting the news that she is...
4  60133  #NoPasaran: Unity demo to oppose the far-right...
[np.int64(0), np.int64(1)]
       0    1
0  15923  OFF
1  27014  NOT
2  30530  NOT
3  13876  NOT
4  60133  OFF
860
0
Checking length of validation set
860 860


In [20]:
original_test = text_array_test

In [21]:
Counter(labels_target_test)

Counter({'TIN': 213, 'NA': 620, 'UNT': 27})

In [22]:
import re

def remove_website(text):
    pattern = re.compile(r'(?i)\b(?:https?://\S+|www\.\S+|\S+\.(com|co|net)\b)')
    return " ".join([word if not pattern.search(word) else "" for word in text.split()])


# Training set
text_array = text_array.apply(lambda text: remove_website(text))
print_text(text_array,0,10)

print("**************************************************************************")

# Validation set
text_array_test = text_array_test.apply(lambda text: remove_website(text))
print_text(text_array_test,0,10)

@USER She should ask a few native Americans what their take on this is.

@USER @USER Go home you’re drunk!!! @USER #MAGA #Trump2020 👊🇺🇸👊 URL

Amazon is investigating Chinese employees who are selling internal data to third-party sellers looking for an edge in the competitive marketplace. URL #Amazon #MAGA #KAG #CHINA #TCOT

@USER Someone should'veTaken" this piece of shit to a volcano. 😂"

@USER @USER Obama wanted liberals &amp; illegals to move into red states

@USER Liberals are all Kookoo !!!

@USER @USER Oh noes! Tough shit.

@USER was literally just talking about this lol all mass shootings like that have been set ups. it’s propaganda used to divide us on major issues like gun control and terrorism

@USER Buy more icecream!!!

@USER Canada doesn’t need another CUCK! We already have enough #LooneyLeft #Liberals f**king up our great country! #Qproofs #TrudeauMustGo

**************************************************************************
#WhoIsQ #WheresTheServer #DumpNike #DECLASF

In [23]:
# Functions for chat word conversion
f = open("/content/drive/My Drive/offenseval/slang.txt", "r")
chat_words_str = f.read()
chat_words_map_dict = {}
chat_words_list = []

for line in chat_words_str.split("\n"):
    if line != "":
        cw = line.split("=")[0]
        cw_expanded = line.split("=")[1]
        chat_words_list.append(cw)
        chat_words_map_dict[cw] = cw_expanded
chat_words_list = set(chat_words_list)

def chat_words_conversion(text):
    new_text = []
    for w in text.split():
        if w.upper() in chat_words_list:
            new_text.append(chat_words_map_dict[w.upper()])
        else:
            new_text.append(w)
    return " ".join(new_text)

In [24]:
# Chat word conversion
# Training set
text_array = text_array.apply(lambda text: chat_words_conversion(text))
print_text(text_array,0,10)
print_text(original,0,10)

print("********************************************************************************")

# Validation set
text_array_test = text_array_test.apply(lambda text: chat_words_conversion(text))
print_text(text_array_test,0,10)

@USER She should ask a few native Americans what their take on this is.

@USER @USER Go home you’re drunk!!! @USER #MAGA #Trump2020 👊🇺🇸👊 URL

Amazon is investigating Chinese employees who are selling internal data to third-party sellers looking for an edge in the competitive marketplace. URL #Amazon #MAGA #KAG #CHINA #TCOT

@USER Someone should'veTaken" this piece of shit to a volcano. 😂"

@USER @USER Obama wanted liberals &amp; illegals to move into red states

@USER Liberals are all Kookoo !!!

@USER @USER Oh noes! Tough shit.

@USER was literally just talking about this Laughing Out Loud all mass shootings like that have been set ups. it’s propaganda used to divide us on major issues like gun control and terrorism

@USER Buy more icecream!!!

@USER Canada doesn’t need another CUCK! We already have enough #LooneyLeft #Liberals f**king up our great country! #Qproofs #TrudeauMustGo

@USER She should ask a few native Americans what their take on this is.

@USER @USER Go home you’re drun

In [25]:
os.chdir("/content/drive/My Drive/offenseval")
print(os.getcwd())

/content/drive/My Drive/offenseval


In [26]:
#Function for emoticon conversion
from emoticons import EMOTICONS

def convert_emoticons(text):
    for emot in EMOTICONS:
        text = re.sub(u'('+emot+')', " ".join(EMOTICONS[emot].replace(",","").split()), text)
    return text


#testing the emoticon function
text = "Hello :-) :-)"
text = convert_emoticons(text)
print(text + "\n")

Hello Happy face smiley Happy face smiley



In [27]:
# Emoticon conversion
# Training set
text_array = text_array.apply(lambda text: convert_emoticons(text))
print_text(text_array,0,10)

print("**********************************************************************************")

# Validation set
text_array_test = text_array_test.apply(lambda text: convert_emoticons(text))
print_text(text_array_test,0,10)

@USER She should ask a few native Americans what their take on this is.

@USER @USER Go home you’re drunk!!! @USER #MAGA #Trump2020 👊🇺🇸👊 URL

Amazon is investigating Chinese employees who are selling internal data to third-party sellers looking for an edge in the competitive marketplace. URL #Amazon #MAGA #KAG #CHINA #TCOT

@USER Someone should'veTaken" this piece of shit to a volcano. 😂"

@USER @USER Obama wanted liberals &amp; illegals to move into red states

@USER Liberals are all Kookoo !!!

@USER @USER Oh noes! Tough shit.

@USER was literally just talking about this Laughing Out Loud all mass shootings like that have been set ups. it’s propaganda used to divide us on major issues like gun control and terrorism

@USER Buy more icecream!!!

@USER Canada doesn’t need another CUCK! We already have enough #LooneyLeft #Liberals f**king up our great country! #Qproofs #TrudeauMustGo

**********************************************************************************
#WhoIsQ #WheresTheSer

In [28]:
# FUnction for removal of emoji
import emoji

def convert_emojis(text):
    text = emoji.demojize(text, delimiters=(" ", " "))
    text = re.sub("_|-"," ",text)
    return text

# Training set
text_array = text_array.apply(lambda text: convert_emojis(text))
print_text(text_array,0,10)

print("**************************************************************************")

# Validation set
text_array_test = text_array_test.apply(lambda text: convert_emojis(text))
print_text(text_array_test,0,10)

@USER She should ask a few native Americans what their take on this is.

@USER @USER Go home you’re drunk!!! @USER #MAGA #Trump2020  oncoming fist  United States  oncoming fist  URL

Amazon is investigating Chinese employees who are selling internal data to third party sellers looking for an edge in the competitive marketplace. URL #Amazon #MAGA #KAG #CHINA #TCOT

@USER Someone should'veTaken" this piece of shit to a volcano.  face with tears of joy "

@USER @USER Obama wanted liberals &amp; illegals to move into red states

@USER Liberals are all Kookoo !!!

@USER @USER Oh noes! Tough shit.

@USER was literally just talking about this Laughing Out Loud all mass shootings like that have been set ups. it’s propaganda used to divide us on major issues like gun control and terrorism

@USER Buy more icecream!!!

@USER Canada doesn’t need another CUCK! We already have enough #LooneyLeft #Liberals f**king up our great country! #Qproofs #TrudeauMustGo

****************************************

In [29]:
os.chdir("/content")
print(os.getcwd())

/content


In [30]:
# Ekphrasis pipe for text pre-processing
def ekphrasis_pipe(sentence):
    cleaned_sentence = " ".join(text_processor.pre_process_doc(sentence))
    return cleaned_sentence

# Training set
text_array = text_array.apply(lambda text: ekphrasis_pipe(text))
print("Training set completed.......")
#Validation set
text_array_test = text_array_test.apply(lambda text: ekphrasis_pipe(text))
print("Test set completed.......")

Training set completed.......
Test set completed.......


In [31]:
print_text(text_array,0,10)
print("************************************************************************")
print_text(text_array_test,0,10)

<user> she should ask a few native americans what their take on this is .

<user> <user> go home you ’ re drunk ! <repeated> <user> <hashtag> maga </hashtag> <hashtag> trump 2020 </hashtag> oncoming fist united states oncoming fist <allcaps> url </allcaps>

amazon is investigating chinese employees who are selling internal data to third party sellers looking for an edge in the competitive marketplace . <allcaps> url </allcaps> <hashtag> amazon </hashtag> <hashtag> maga </hashtag> <hashtag> kag </hashtag> <hashtag> china </hashtag> <hashtag> tcot </hashtag>

<user> someone should havetaken " this piece of shit to a volcano . face with tears of joy "

<user> <user> obama wanted liberals & illegals to move into red states

<user> liberals are all kookoo ! <repeated>

<user> <user> oh noes ! tough shit .

<user> was literally just talking about this laughing out loud all mass shootings like that have been set ups . it ’ s propaganda used to divide us on major issues like gun control and te

In [32]:
# Removing unnecessary punctuations
PUNCT_TO_REMOVE = "\"$%&'()+,-./;=[\]^_`{|}~"
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

# Training set
text_array = text_array.apply(lambda text: remove_punctuation(text))
print_text(text_array,0,10)

print("********************************************************************")

# Validation set
text_array_test = text_array_test.apply(lambda text: remove_punctuation(text))
print_text(text_array_test,0,10)

<user> she should ask a few native americans what their take on this is 

<user> <user> go home you ’ re drunk ! <repeated> <user> <hashtag> maga <hashtag> <hashtag> trump 2020 <hashtag> oncoming fist united states oncoming fist <allcaps> url <allcaps>

amazon is investigating chinese employees who are selling internal data to third party sellers looking for an edge in the competitive marketplace  <allcaps> url <allcaps> <hashtag> amazon <hashtag> <hashtag> maga <hashtag> <hashtag> kag <hashtag> <hashtag> china <hashtag> <hashtag> tcot <hashtag>

<user> someone should havetaken  this piece of shit to a volcano  face with tears of joy 

<user> <user> obama wanted liberals  illegals to move into red states

<user> liberals are all kookoo ! <repeated>

<user> <user> oh noes ! tough shit 

<user> was literally just talking about this laughing out loud all mass shootings like that have been set ups  it ’ s propaganda used to divide us on major issues like gun control and terrorism

<user> b

In [33]:
# print_text(text_array,3550,3555)
print_text(original,9540,9555)

@USER i think its because he is remaining this pure sweet thing

@USER @USER You think they’ll “get” violent? Antifa already is..... I do agree it will get worse.  That’s why we all need to learn self defense and get concealed carry approved w/insurance.

@USER So what ur saying is that it is in fact chicken shit and you are eating chicken shit URL

@USER I blame @USER for inciting violence and she should be held accountable! She is a despicable disgrace.

@USER @USER My sister is in G’ville. Says she is going to text you

@USER Here she is........ URL

@USER Liberals with their lies again.

@USER Police state tyrant liberals LOVE the word dangerous" when gaslighting their sheep."

@USER @USER @USER @USER It to t dd

@USER @USER @USER @USER Obama will be remembered as the man that destroy libya. Imagine if mi6 or cia give blm or antifa guns and wages war vs trump. Then they control the msm and say trump use chemical weapon to legalize their coup. The blood is on obama hand.

@USER @USE

In [34]:
# Finding length of longest array
maxLen = len(max(text_array,key = lambda text: len(text.split(" "))).split(" "))
print(maxLen)

159


In [35]:
u = lambda text: len(text.split(" "))
sentence_lengths = []
for x in text_array:
    sentence_lengths.append(u(x))
print(sorted(sentence_lengths)[-100:])
print(len(sentence_lengths))

[80, 80, 80, 81, 81, 81, 81, 81, 81, 81, 82, 82, 82, 82, 82, 82, 83, 83, 83, 83, 84, 84, 84, 84, 85, 85, 85, 85, 85, 86, 86, 86, 86, 87, 87, 87, 87, 87, 87, 87, 88, 88, 88, 88, 88, 89, 89, 89, 89, 89, 90, 91, 91, 91, 91, 92, 92, 92, 92, 92, 93, 93, 93, 93, 94, 94, 94, 95, 95, 96, 96, 97, 97, 97, 98, 99, 100, 100, 100, 101, 101, 102, 103, 103, 103, 104, 105, 107, 108, 108, 109, 109, 111, 111, 112, 115, 116, 117, 119, 159]
13240


<h2>Text pre-processing complete</h2>

In [36]:
# Count of each label in dataset
from collections import Counter

# Printing training set counts for analysis
print("Elements: ",set(labels))
print("Length: ",len(labels))
print(Counter(labels))

print("**************************************************************************")

# Printing validation set counts for analysis
print("Elements: ",set(labels_test))
print("Length: ",len(labels_test))
print(Counter(labels_test))

Elements:  {'NOT', 'OFF'}
Length:  13240
Counter({'NOT': 8840, 'OFF': 4400})
**************************************************************************
Elements:  {'NOT', 'OFF'}
Length:  860
Counter({'NOT': 620, 'OFF': 240})


In [37]:
Y = []
Y_test = []

# Training set
for i in range(0,len(labels)):
    if(labels[i] == "OFF"):
        Y.append(0)
    if(labels[i] == "NOT"):
        Y.append(1)

# Validation set
for i in range(0,len(labels_test)):
    if(labels_test[i] == "OFF"):
        Y_test.append(0)
    if(labels_test[i] == "NOT"):
        Y_test.append(1)

In [38]:
Y_target = []
Y_target_test = []

# Training set
for i in range(0,len(labels_target)):
    if(labels_target[i] == "NA"):
        Y_target.append(0)
    if(labels_target[i] == "TIN"):
        Y_target.append(1)
    if(labels_target[i] == "UNT"):
        Y_target.append(2)

# Validation set
for i in range(0,len(labels_target_test)):
    if(labels_target_test[i] == "NA"):
        Y_target_test.append(0)
    if(labels_target_test[i] == "TIN"):
        Y_target_test.append(1)
    if(labels_target_test[i] == "UNT"):
        Y_target_test.append(2)

In [39]:
# Testing the conversion into integers
for i in range(200,210):
    print(text_array_test[i])
    print(labels_test[i],Y_test[i])
    print(labels_target_test[i],Y_target_test[i])
    print()

<hashtag> antifa <hashtag> demo in memory of <hashtag> killah p <hashtag>  <hashtag> greece <hashtag> <allcaps> url <allcaps>
NOT 1
NA 0

<user> these are all coranated attacks from idiot loons ! they will get worse much worse before the election ! can you imagine idiots who want to get rid of the economy and their <number> along with interest rates just because he is not a polished politician ? idiots !
OFF 0
TIN 1

<user> <user> a <elongated> she is so stinking cute ! how old is she now ?
OFF 0
TIN 1

<hashtag> benghazi <hashtag> <hashtag> haiti <hashtag> <hashtag> uranium one <hashtag> <hashtag> seth rich <hashtag> what more does the justice department need ? no double standards ! just because you are rich and powerful does not mean you can rob  seal and murder ! <hashtag> drain the swamp <hashtag> <hashtag> hillary for prison <hashtag> <hashtag> vote red <hashtag> <hashtag> maga <hashtag> <allcaps> url <allcaps>
OFF 0
UNT 2

<hashtag> black <hashtag> eyed peas tackle gun control  s

In [40]:
# Verifying train set
X = np.asarray(list(text_array))
Y = np.asarray(list(Y))
Y_target = np.asarray(list(Y_target))
print(type(X))
print(type(Y))
print(type(Y_target))
print(np.shape(X),np.shape(Y),np.shape(Y_target))

# Verifying validation set
X_test = np.asarray(list(text_array_test))
Y_test = np.asarray(list(Y_test))
Y_target_test = np.asarray(list(Y_target_test))
print(type(X_test))
print(type(Y_test))
print(type(Y_target_test))
print(np.shape(X_test),np.shape(Y_test),np.shape(Y_target_test))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(13240,) (13240,) (13240,)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(860,) (860,) (860,)


In [41]:
print(Counter(Y))
print(Counter(Y_test))

Counter({np.int64(1): 8840, np.int64(0): 4400})
Counter({np.int64(1): 620, np.int64(0): 240})


In [42]:
print(X_test[0])
print(Y_test[0])
print(labels_test[0])
print(Y_target_test[0])
print(labels_target_test[0])

<hashtag> who is q <hashtag> <hashtag> wheres the server <hashtag> <hashtag> dump nike <hashtag> <hashtag> declasfisa <hashtag> democrats support antifa  muslim brotherhood  ms13  <allcaps> isis <allcaps>  pedophilia  child trafficking  taxpayer funded abortion ’ s  election fraud  sedition and treason ! <repeated> <hashtag> lock them all up <hashtag> <hashtag> wwg 1 wga <hashtag> <hashtag> q anon <hashtag> ⁦ <user> <allcaps> url <allcaps>
0
OFF
1
TIN


<h2>Shuffling training and validation data</h2>

In [43]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [44]:
print(Counter(labels))
print(Counter(labels_test))
print(Counter(labels_target))
print(Counter(labels_target_test))

Counter({'NOT': 8840, 'OFF': 4400})
Counter({'NOT': 620, 'OFF': 240})
Counter({'NA': 8840, 'TIN': 3876, 'UNT': 524})
Counter({'NA': 620, 'TIN': 213, 'UNT': 27})


In [45]:
# Converting to one hot vectors
def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)] #u[Y] helps to index each element of Y index at u. U here is a class array
    return Y

In [46]:
Y_oh_train = convert_to_one_hot(np.array(Y), C = 2)
Y_oh_test = convert_to_one_hot(np.array(Y_test), C = 2)

Y_oh_target_train = convert_to_one_hot(np.array(Y_target), C = 3)
Y_oh_target_test = convert_to_one_hot(np.array(Y_target_test), C = 3)
print(np.shape(Y_oh_train))
print(np.shape(Y_oh_target_test))
index = 0
print(labels[index], Y[index], "is converted into one hot", Y_oh_train[index])
print(labels_target[index], Y_target[index], "is converted into one hot", Y_oh_target_train[index])

(13240, 2)
(860, 3)
OFF 0 is converted into one hot [1. 0.]
UNT 2 is converted into one hot [0. 0. 1.]


<h2>Model using RoBERTa</h2>

In [47]:
!pip install tensorflow


In [48]:
import tensorflow as tf
import os
import numpy as np
import pandas as pd
import string
from nltk.corpus import stopwords
import re
import os
from collections import Counter

In [49]:
pip install tf-keras

In [50]:
from transformers import RobertaTokenizerFast, TFRobertaModel, TFBertModel, BertTokenizerFast, ElectraTokenizerFast, TFElectraModel, AlbertTokenizerFast, TFAlbertModel, XLNetTokenizerFast, TFXLNetModel, MPNetTokenizerFast, TFMPNetModel
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import RepeatVector, Concatenate, Dense, Activation, Dot, BatchNormalization, Dropout

from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

In [51]:
print(tf.__version__)

2.18.0


In [52]:
import tensorflow as tf

if tf.config.list_physical_devices('GPU'):
    print("✅ GPU is available and ready to use.")
    print(tf.config.list_physical_devices('GPU'))
else:
    print("❌ GPU not detected. Please recheck runtime settings.")


✅ GPU is available and ready to use.
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [53]:
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [54]:
X = list(X)
X_test = list(X_test)

In [55]:
model_train_x, model_val_x, Y_train, Y_val = train_test_split(X, Y, test_size=0.05, random_state=44)

In [56]:
train_encodings = tokenizer(model_train_x, max_length=100, truncation=True, padding="max_length", return_tensors='tf')
val_encodings = tokenizer(model_val_x, max_length=100, truncation=True, padding="max_length", return_tensors='tf')
test_encodings = tokenizer(X_test, max_length=100, truncation=True, padding="max_length", return_tensors='tf')

In [57]:
cluster_encodings = tokenizer(X, max_length=100, truncation=True, padding="max_length", return_tensors='tf')

In [58]:
print(np.shape(train_encodings["input_ids"]))
print(np.shape(val_encodings["input_ids"]))
print(np.shape(test_encodings["input_ids"]))
print(np.shape(cluster_encodings["input_ids"]))

(12578, 100)
(662, 100)
(860, 100)
(13240, 100)


In [59]:
print(train_encodings["input_ids"][0])
print("***************************************************************************")
print(val_encodings["input_ids"][0])
print("***************************************************************************")
print(test_encodings["input_ids"][0])

tf.Tensor(
[    0 41552 12105 15698 28696 12105 15698    79  7533  6245  3867    63
     7   492    10 18066   995     7   120   658    50  9876 15247    24
  1302  1437     2     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1], shape=(100,), dtype=int32)
***************************************************************************
tf.Tensor(
[    0 41552 12105 15698 28696 12105 15698    47  2369   101     5 28696
  1250 25879 15698   295   763 28696  1250 25879 15698   584    24    44
    27   579   350  1010     7  1067    59  1751   797  1437   114    51
    58   593     7   512    25    47   224   

<h3> Subtask A</h3>

In [60]:
def Offense_classifier(input_shape):
    """
    Function creating the model's graph.

    Arguments:
    input_shape -- shape of the input,(max_len,)
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 100-dimensional vector representation
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (13 million words)

    Returns:
    model -- a model instance in Keras
    """
    model = TFRobertaModel.from_pretrained('roberta-base')
    layer = model.layers[0]

    # Define sentence_indices as the input of the graph, it should be of shape input_shape and dtype 'int32' (as it contains indices).
    inputs = keras.Input(shape=input_shape, dtype='int32')
    input_masks = keras.Input(shape=input_shape, dtype='int32')

    embeddings = layer([inputs, input_masks])[1]

    X = BatchNormalization(momentum=0.99, epsilon=0.001, center=True, scale=True)(embeddings)

    # Add dropout with a probability of 0.1
    X = Dropout(0.1)(X)

    X = Dense(128,activation='elu',kernel_regularizer=keras.regularizers.l2(0.001))(X)

    X = Dense(32,activation='elu',kernel_regularizer=keras.regularizers.l2(0.001))(X)

    X = Dense(3,activation='elu',kernel_regularizer=keras.regularizers.l2(0.01))(X)

    X = Dense(32,activation='elu',kernel_regularizer=keras.regularizers.l2(0.001))(X)

    X = BatchNormalization(momentum=0.99, epsilon=0.001, center=True, scale=True)(X)

    X = Dense(128,activation='elu',kernel_regularizer=keras.regularizers.l2(0.001))(X)

    X = Dense(1,activation='linear',kernel_regularizer=keras.regularizers.l2(0.01))(X)

    # Add a sigmoid activation
    X = Activation('sigmoid')(X)

    # Create Model instance which converts sentence_indices into X.
    model = keras.Model(inputs=[inputs,input_masks], outputs=[X])


    return model

In [70]:
from tensorflow.keras import regularizers
from transformers import TFRobertaModel
import tensorflow as tf
from tensorflow.keras import layers

def Offense_classifier(input_shape):
    """
    Function creating the model's graph.

    Arguments:
    input_shape -- shape of the input,(max_len,)

    Returns:
    model -- a model instance in Keras
    """
    model = TFRobertaModel.from_pretrained('roberta-base')
    layer = model.layers[0]

    # Define sentence_indices as the input of the graph, it should be of shape input_shape and dtype 'int32' (as it contains indices).
    inputs = tf.keras.Input(shape=input_shape, dtype='int32')
    input_masks = tf.keras.Input(shape=input_shape, dtype='int32')

    embeddings = layer([inputs, input_masks])[1]

    X = layers.BatchNormalization(momentum=0.99, epsilon=0.001, center=True, scale=True)(embeddings)

    # Add dropout with a probability of 0.1
    X = layers.Dropout(0.1)(X)

    X = layers.Dense(128, activation='elu', kernel_regularizer=regularizers.l2(0.001))(X)

    X = layers.Dense(32, activation='elu', kernel_regularizer=regularizers.l2(0.001))(X)

    X = layers.Dense(3, activation='elu', kernel_regularizer=regularizers.l2(0.01))(X)

    X = layers.Dense(32, activation='elu', kernel_regularizer=regularizers.l2(0.001))(X)

    X = layers.BatchNormalization(momentum=0.99, epsilon=0.001, center=True, scale=True)(X)

    X = layers.Dense(128, activation='elu', kernel_regularizer=regularizers.l2(0.001))(X)

    X = layers.Dense(1, activation='linear', kernel_regularizer=regularizers.l2(0.01))(X)

    # Add a sigmoid activation
    X = layers.Activation('sigmoid')(X)

    # Create Model instance which converts sentence_indices into X.
    model = tf.keras.Model(inputs=[inputs, input_masks], outputs=[X])

    return model


In [61]:
strategy = tf.distribute.TPUStrategy(resolver)

NameError: name 'resolver' is not defined

In [62]:
class EvaluationMetric(keras.callbacks.Callback):

    def __init__(self, trial_encodings, trial_masks, Y_test):
        super(EvaluationMetric, self).__init__()
        self.trial_encodings = trial_encodings
        self.trial_masks = trial_masks
        self.Y_test = Y_test

    def on_epoch_begin(self, epoch, logs={}):
        print("\nTraining...")

    def on_epoch_end(self, epoch, logs={}):
        print("\nEvaluating...")
        trial_prediction = self.model.predict([self.trial_encodings,self.trial_masks])

        pred = []
        for i in range(0,len(self.Y_test)):
            num = trial_prediction[i]
            if(num > 0.5):
              num = 1
            else:
              num = 0
            pred.append(num)

        from sklearn.metrics import classification_report
        print(classification_report(Y_test, pred, digits=3))

evaluation_metric = EvaluationMetric(test_encodings["input_ids"], test_encodings["attention_mask"], Y_test)

In [63]:
with strategy.scope():
    model = Offense_classifier((100,))
    optimizer = keras.optimizers.Adam(learning_rate=2e-5)
    loss_fun = [
          tf.keras.losses.BinaryCrossentropy()
    ]
    metric = ['acc']
    model.compile(optimizer=optimizer, loss=loss_fun, metrics=metric)

NameError: name 'strategy' is not defined

In [64]:
model.summary()

NameError: name 'model' is not defined

In [65]:
neg, pos = np.bincount(Y)
total = neg + pos
print('Examples:\n    Total: {}\n    Positive: {} ({:.2f}% of total)\n'.format(
    total, pos, 100 * pos / total))

Examples:
    Total: 13240
    Positive: 8840 (66.77% of total)



In [66]:
class_weight = {}
maxi = max(neg, pos)
weight_for_0 = (maxi / (maxi + neg))
weight_for_1 = (maxi / (maxi + pos))

class_weight = {0: weight_for_0, 1: weight_for_1}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))

Weight for class 0: 0.67
Weight for class 1: 0.50


In [67]:
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(filepath='/content/olid_roberta.{epoch:03d}.h5',
                                 monitor='val_acc',
                                 verbose=1,
                                 save_weights_only=True,
                                 period=1)

TypeError: ModelCheckpoint.__init__() got an unexpected keyword argument 'period'

In [ ]:
print(Counter(Y))
print(Counter(Y_test))

In [ ]:
print(Counter(Y_train))
print(Counter(Y_val))

In [ ]:
print(len(train_encodings["input_ids"]),len(val_encodings["input_ids"]))

In [ ]:
# val 0.05
history = model.fit(
    x = [train_encodings["input_ids"], train_encodings["attention_mask"]],
    y = Y_train,
    validation_data = ([val_encodings["input_ids"],val_encodings["attention_mask"]],Y_val),
    callbacks = [evaluation_metric, checkpoint],
    batch_size = 64,
    shuffle=True,
    epochs=6,
    class_weight = class_weight
)

In [72]:
input_ids = np.array(input_ids)  # Shape should be (num_samples, max_length)
attention_masks = np.array(attention_masks)  # Shape should be (num_samples, max_length)
labels = np.array(labels)  # Shape should be (num_samples,)

# Now check shapes
print(f"input_ids shape: {input_ids.shape}")
print(f"attention_masks shape: {attention_masks.shape}")
print(f"labels shape: {labels.shape}")

# Ensure they all match, then fit the model
model.fit([input_ids, attention_masks], labels, epochs=3, batch_size=32)


input_ids shape: (2, 8)
attention_masks shape: (2, 8)
labels shape: (13240,)


ValueError: Data cardinality is ambiguous:
  x sizes: 2, 2
  y sizes: 13240
Make sure all arrays contain the same number of samples.

In [78]:
from transformers import RobertaTokenizer
import numpy as np

# Load the tokenizer for RoBERTa
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Example texts (this should be replaced with your actual dataset)
texts = ["This is a sample sentence."] * 13240  # Replace with your actual data

# Tokenize the texts with padding and truncation to max length of 128
inputs = tokenizer(texts, padding='max_length', truncation=True, max_length=128, return_tensors='np')

# Extract tokenized inputs
input_ids = inputs['input_ids']
attention_masks = inputs['attention_mask']

# Verify the shapes of the tokenized inputs
print(f"input_ids shape: {input_ids.shape}")  # Should print (13240, 128)
print(f"attention_masks shape: {attention_masks.shape}")  # Should print (13240, 128)

# Labels for training (assuming binary classification task)
labels = np.random.randint(0, 2, size=(13240,))

# Verify labels shape
print(f"labels shape: {labels.shape}")  # Should print (13240,)

# Define your model (ensure the input shape matches)
model = Offense_classifier(input_shape=(128,))  # Ensure the sequence length matches
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Fit the model
model.fit([input_ids, attention_masks], labels, epochs=3, batch_size=32)


input_ids shape: (13240, 128)
attention_masks shape: (13240, 128)
labels shape: (13240,)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['lm_head.layer_norm.bias', 'roberta.embeddings.position_ids', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infe

Epoch 1/3
414/414 [==============================] - 372s 794ms/step - loss: 0.6936 - accuracy: 0.4997
Epoch 2/3
414/414 [==============================] - 327s 790ms/step - loss: 0.6936 - accuracy: 0.4997
Epoch 3/3
414/414 [==============================] - 328s 792ms/step - loss: 0.6936 - accuracy: 0.4997


In [1]:
import pandas as pd

label_map = {'not sexist': 0, 'sexist': 1}

train_df = pd.read_csv("train.csv")
val_df = pd.read_csv("validate.csv")
test_df = pd.read_csv("test.csv")

# Assuming the text is in 'text' and label in 'label_sexist'
train_texts, train_labels = train_df['text'].tolist(), train_df['label_sexist'].map(label_map).values
val_texts, val_labels = val_df['text'].tolist(), val_df['label_sexist'].map(label_map).values
test_texts, test_labels = test_df['text'].tolist(), test_df['label_sexist'].map(label_map).values


In [2]:
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

def tokenize_data(texts):
    return tokenizer(
        texts, padding='max_length', truncation=True, max_length=128, return_tensors="tf"
    )

train_encodings = tokenize_data(train_texts)
val_encodings = tokenize_data(val_texts)
test_encodings = tokenize_data(test_texts)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
print("Original test_labels:", test_labels[:10])



Original test_labels: [0 1 0 1 1 0 0 0 0 1]


In [4]:
import numpy as np
import tensorflow as tf
from transformers import TFRobertaModel

# Load model once
roberta = TFRobertaModel.from_pretrained('roberta-base')

def extract_features(encodings, batch_size=32):
    input_ids = encodings['input_ids']
    attention_mask = encodings['attention_mask']

    all_outputs = []
    for i in range(0, len(input_ids), batch_size):
        input_batch = input_ids[i:i+batch_size]
        mask_batch = attention_mask[i:i+batch_size]

        outputs = roberta(input_batch, attention_mask=mask_batch, training=False)
        cls_embeddings = outputs.last_hidden_state[:, 0, :]  # CLS token
        all_outputs.append(cls_embeddings.numpy())

    return np.vstack(all_outputs)

# Apply batched extraction
train_output = extract_features(train_encodings, batch_size=32)
val_output = extract_features(val_encodings, batch_size=32)
test_output = extract_features(test_encodings, batch_size=32)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['roberta.embeddings.position_ids', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infe

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense

model = Sequential([
    Input(shape=(train_output.shape[1],)),
    Dense(128, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])


In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential([
    Dense(128, activation='relu', input_shape=(train_output.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(train_output, np.array(train_labels), epochs=5, validation_data=(val_output, np.array(val_labels)))


Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


438/438 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.7583 - loss: 0.5256 - val_accuracy: 0.7905 - val_loss: 0.4447
Epoch 2/5
438/438 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.7985 - loss: 0.4362 - val_accuracy: 0.7920 - val_loss: 0.4291
Epoch 3/5
438/438 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8067 - loss: 0.4260 - val_accuracy: 0.7920 - val_loss: 0.4302
Epoch 4/5
438/438 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7966 - loss: 0.4324 - val_accuracy: 0.7985 - val_loss: 0.4240
Epoch 5/5
438/438 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8062 - loss: 0.4261 - val_accuracy: 0.7935 - val_loss: 0.4257


In [7]:
from sklearn.metrics import classification_report
import numpy as np

# Predict on the validation or test set (choose accordingly)
predictions = model.predict(val_output)  # For validation, or use test_output for test set
predictions = (predictions > 0.5).astype(int)  # Convert probabilities to binary predictions (0 or 1)

# If you want a classification report for the validation set:
print("Classification Report for Validation Set:")
print(classification_report(np.array(val_labels), predictions))

# If you want a classification report for the test set:
# print("Classification Report for Test Set:")
# print(classification_report(np.array(test_labels), predictions))


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
Classification Report for Validation Set:
              precision    recall  f1-score   support

           0       0.87      0.85      0.86      1514
           1       0.57      0.62      0.59       486

    accuracy                           0.79      2000
   macro avg       0.72      0.73      0.73      2000
weighted avg       0.80      0.79      0.80      2000



In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pandas as pd

# Recreate clean training/test sets using proper NaN handling
train_mask = pd.notna(train_labels)
train_output_clean = train_output[train_mask]
train_labels_clean = train_labels[train_mask]

test_mask = pd.notna(test_labels)
test_output_clean = test_output[test_mask]
test_labels_clean = test_labels[test_mask]

# Now train the Random Forest
rf = RandomForestClassifier(n_estimators=100)
rf.fit(train_output_clean, train_labels_clean)
preds = rf.predict(test_output_clean)

accuracy = accuracy_score(test_labels_clean, preds)
print("Random Forest Accuracy on Test Set:", accuracy)


Random Forest Accuracy on Test Set: 0.778


In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rf = RandomForestClassifier(n_estimators=100)
rf.fit(train_output, train_labels)
preds = rf.predict(test_output)

accuracy = accuracy_score(test_labels, preds)
print("Random Forest Accuracy on Test Set:", accuracy)


ValueError: Input y contains NaN.

In [ ]:
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture

# Apply on RoBERTa features
kmeans = KMeans(n_clusters=5).fit(train_output.numpy())
gmm = GaussianMixture(n_components=5).fit(train_output.numpy())

# Concatenate cluster assignments
import numpy as np

train_features = np.concatenate([
    train_output.numpy(),
    kmeans.predict(train_output.numpy()).reshape(-1, 1),
    gmm.predict(train_output.numpy()).reshape(-1, 1)
], axis=1)

val_features = np.concatenate([
    val_output.numpy(),
    kmeans.predict(val_output.numpy()).reshape(-1, 1),
    gmm.predict(val_output.numpy()).reshape(-1, 1)
], axis=1)

test_features = np.concatenate([
    test_output.numpy(),
    kmeans.predict(test_output.numpy()).reshape(-1, 1),
    gmm.predict(test_output.numpy()).reshape(-1, 1)
], axis=1)


<h4>Training Curves</h4>

In [ ]:
history = history
import matplotlib.pyplot as plt
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='lower right')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='lower right')
plt.show()

In [ ]:
# model.load_weights("/content/drive/MyDrive/OLID Transformer weights/olid_roberta(0.05).003.h5")

In [ ]:
# model.save_weights("/content/drive/MyDrive/OLID Transformer weights/olid_roberta(0.05).003.h5")

<h4>Test Set Statistics</h4>

In [ ]:
answer = model.predict([test_encodings["input_ids"], test_encodings["attention_mask"]])

In [ ]:
pred = []
sample = df_test_text["tweet"]
count = 0
for i in range(0,len(X_test)):

    num = answer[i]
    if(num >= 0.5):
      num = 1
    else:
      num = 0
    pred.append(num)
print(count)

In [ ]:
con_mat = tf.math.confusion_matrix(labels=Y_test, predictions=pred)
print(con_mat)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
figure = plt.figure(figsize=(8, 8))
sns.set(font_scale=1.75)
sns.heatmap(con_mat, annot=True,cmap=plt.cm.viridis,fmt='d', xticklabels=["Offensive","Not Offensive"], yticklabels=["Offensive","Not Offensive"],annot_kws={"size": 15})
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
from sklearn.metrics import f1_score, classification_report

In [ ]:
f1_score(Y_test, pred, average='macro')

In [ ]:
print(classification_report(Y_test, pred, target_names=["offensive", "not offensive"], digits=3))

<h3>Train set analysis</h3>

In [ ]:
answer_train = model.predict([cluster_encodings["input_ids"], cluster_encodings["attention_mask"]])

In [ ]:
pred = []
sample = original
count = 0
for i in range(0,len(Y)):

    num = answer_train[i]
    lol = num
    if(num > 0.5):
      num = 1
    else:
      num = 0
    pred.append(num)
    if(num != Y[i] and Y[i] == 0 and lol >=0.8):
        print(answer_train[i])
        print("Original label: ",labels[i])
        print("Without pre-processing: ",sample[i])
        print("With pre-processing: ",X[i])
        lol = ""
        count += 1

        if(num == 0):
            lol = "Offensive"
        if(num == 1):
            lol = "Not Offensive"
        print("Predicted: " + lol)
        print()

print(count)

<h3>Training examination</h3>

In [ ]:
import plotly
import plotly.graph_objs as go
import plotly.express as px

In [ ]:
# 3 neuron output
model.layers[-6].name

In [ ]:
cluster_dense_3 = keras.Model(inputs=model.input, outputs=model.layers[-6].output)
with strategy.scope():
    cluster_3 = cluster_dense_3.predict([cluster_encodings["input_ids"], cluster_encodings["attention_mask"]])

In [ ]:
pred_train = []
temp = 0

for i in range(0,len(Y)):

    num = answer_train[i]

    if(num >= 0.5):
      num = 1
    else:
      num = 0
    pred_train.append(num)

print(temp)

In [ ]:
flag = []
count = 0

x_ = []
y_ = []
z_ = []


for i in range(0,len(Y)):
    count = count + 1
    x_.append(cluster_3[i][1])
    y_.append(cluster_3[i][0])
    z_.append(cluster_3[i][2])

    if( answer_train[i] > 0.3 and answer_train[i] < 0.7 ):
        flag.append(2)

    if( answer_train[i] > 0 and answer_train[i] <= 0.3 ):
        flag.append(0)

    if( answer_train[i] >= 0.7 and answer_train[i] < 1 ):
        flag.append(1)


In [ ]:
Counter(flag)

In [ ]:
con_mat = tf.math.confusion_matrix(labels=Y, predictions=pred_train)
print(con_mat)

In [ ]:
pred_colour = []
for i in range(0,len(flag)):
    if flag[i] == 2:
      pred_colour.append("Neutral")
    if flag[i] == 1:
      pred_colour.append("Not Offensive")
    if flag[i] == 0:
      pred_colour.append("Offensive")

test_df = pd.DataFrame({'x':x_, 'y':y_, 'z':z_, 'Labels':pred_colour})

fig = px.scatter_3d(test_df, x='x', y='y', z='z', color='Labels')
fig.update_traces(
    marker={
        'size': 1,
        'opacity': 0.7,
        'colorscale' : 'Oryel',
    }
)
fig.update_layout(legend= {'itemsizing': 'constant', 'font_size':18}, font_size=15, scene_aspectmode='cube')
fig.update_layout(width = 750, height = 500)
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))

In [ ]:
pred_colour = []
for i in range(0,len(flag)):
    if pred_train[i] == 1:
      pred_colour.append("Not Offensive")
    if pred_train[i] == 0:
      pred_colour.append("Offensive")

test_df = pd.DataFrame({'X':x_, 'Y':y_, 'Z':z_, 'Labels:':pred_colour})

fig = px.scatter_3d(test_df, x='X', y='Y', z='Z', color='Labels:')
fig.update_traces(
    marker={
        'size': 1,
        'opacity': 1,
        'colorscale' : 'rainbow',
    }
)
fig.update_layout(legend= {'itemsizing': 'constant'}, font_size=14, scene_aspectmode='cube')
fig.update_layout(width = 850, height = 750)
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))

<h3>Traning examination end</h3>

<h1>CLUSTERING</h1>

<h3>RoBERTa PLM layer</h3>

In [ ]:
model.layers[-8].name

In [ ]:
cluster_bert = keras.Model(inputs=model.input, outputs=model.layers[-8].output)
with strategy.scope():
    cl_bert = cluster_bert.predict([cluster_encodings["input_ids"], cluster_encodings["attention_mask"]])

In [ ]:
len(cl_bert)

In [ ]:
flag_bert = []
count = 0

x_bert = []
y_bert = []
z_bert = []


for i in range(0,len(Y)):
    count = count + 1
    x_bert.append(cl_bert[i][0])
    y_bert.append(cl_bert[i][1])
    z_bert.append(cl_bert[i][2])

    if( answer_train[i] > 0.28 and answer_train[i] < 0.8 ):
        flag_bert.append(2)

    if( answer_train[i] > 0 and answer_train[i] <= 0.28 ):
        flag_bert.append(0)

    if( answer_train[i] >= 0.8 and answer_train[i] < 1 ):
        flag_bert.append(1)

print(count)

<p>k-means PLM layer</p>

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
kmeans_bert = KMeans(n_clusters=3, random_state=44).fit(cl_bert)
y_kmeans_bert = kmeans_bert.predict(cl_bert)

In [ ]:
Counter(y_kmeans_bert)

In [ ]:
Counter(flag_bert)

In [ ]:
# 1 index values are offensive
# 0 index values are not offensive
# 2 index values are neutral

count = 0
for i in range(0,len(flag_bert)):
      if flag_bert[i] == 2 and y_kmeans_bert[i] == 2:
        count = count + 1
print(count)

In [ ]:
for i in range(0,len(flag_bert)):
    if(y_kmeans_bert[i] == 0):
      y_kmeans_bert[i] = 1
    elif(y_kmeans_bert[i] == 1):
      y_kmeans_bert[i] = 0
    else:
      y_kmeans_bert[i] = 2

In [ ]:
flag_bert = []
count = 0

x_bert = []
y_bert = []
z_bert = []


for i in range(0,len(Y)):
    count = count + 1
    x_bert.append(cl_bert[i][0])
    y_bert.append(cl_bert[i][1])
    z_bert.append(cl_bert[i][2])

    if( answer_train[i] > 0.3 and answer_train[i] < 0.7 ):
        flag_bert.append(2)

    if( answer_train[i] > 0 and answer_train[i] <= 0.3 ):
        flag_bert.append(0)

    if( answer_train[i] >= 0.7 and answer_train[i] < 1 ):
        flag_bert.append(1)


print(count)

In [ ]:
Counter(flag_bert)

In [ ]:
con_mat = tf.math.confusion_matrix(labels=flag_bert, predictions=y_kmeans_bert)
print(con_mat)

In [ ]:
import sklearn
print(sklearn.metrics.classification_report(flag_bert, y_kmeans_bert, output_dict=False, digits=3))

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cosine

In [ ]:
centers_bert = kmeans_bert.cluster_centers_

In [ ]:
svns_off = []
for i in range(0,len(Y)):
    off = cosine(cl_bert[i], centers_bert[1])/2
    svns_off.append(1-off)
print(len(svns_off))

In [ ]:
svns_noff = []
for i in range(0,len(Y)):
    noff = cosine(cl_bert[i], centers_bert[0])/2
    svns_noff.append(1-noff)
print(len(svns_noff))

In [ ]:
svns_neu = []
for i in range(0,len(Y)):
    neu = cosine(cl_bert[i], centers_bert[2])/2
    svns_neu.append(1-neu)
print(len(svns_neu))

<h5>k-means PLM plot</h5>

In [ ]:
import plotly
import plotly.graph_objs as go
import plotly.express as px

In [ ]:
pred_colour = []
for i in range(0,len(y_kmeans_bert)):
    if y_kmeans_bert[i] == 2:
      pred_colour.append("Neutral")
    if y_kmeans_bert[i] == 1:
      pred_colour.append("Not Offensive")
    if y_kmeans_bert[i] == 0:
      pred_colour.append("Offensive")

test_df = pd.DataFrame({'SVNS Offensive':svns_off, 'SVNS Not Offensive':svns_noff, 'SVNS Neutral':svns_neu, 'Labels:':pred_colour})

fig = px.scatter_3d(test_df, x='SVNS Offensive', y='SVNS Not Offensive', z='SVNS Neutral', color='Labels:')
fig.update_traces(
    marker={
        'size': 1,
        'opacity': 1,
        'colorscale' : 'viridis',
    },
)
fig.update_layout(legend= {'itemsizing': 'constant', 'font_size':18}, font_size=15, scene_aspectmode='cube')
fig.update_layout(width = 850, height = 750)
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))

In [ ]:
pred_krobert = []
for i in range(0,len(Y_test)):
    if(svns_off[i] > svns_noff[i]):
        pred_krobert.append(0)
    else:
        pred_krobert.append(1)
print(classification_report(Y_test, pred_krobert, output_dict=False, digits=3))

In [ ]:
con_mat = tf.math.confusion_matrix(labels=Y_test, predictions=pred_krobert)
print(con_mat)

<p> GMM model PLM layer</p>

In [ ]:
from sklearn.mixture import GaussianMixture

In [ ]:
gmm_bert = GaussianMixture(n_components=3, random_state = 44).fit(cl_bert)

In [ ]:
mean_bert = gmm_bert.means_
cov_bert = gmm_bert.covariances_
print(np.shape(mean_bert))
print(np.shape(cov_bert))

In [ ]:
labels_bert = gmm_bert.predict(cl_bert)

In [ ]:
flag_bert = []
count = 0

x_bert = []
y_bert = []
z_bert = []


for i in range(0,len(X)):
    count = count + 1
    x_bert.append(cl_bert[i][0])
    y_bert.append(cl_bert[i][1])
    z_bert.append(cl_bert[i][2])

    if( answer_train[i] > 0.28 and answer_train[i] < 0.8 ):
        flag_bert.append(2)

    if( answer_train[i] > 0 and answer_train[i] <= 0.28 ):
        flag_bert.append(0)

    if( answer_train[i] >= 0.8 and answer_train[i] < 1 ):
        flag_bert.append(1)

print(count)

In [ ]:
Counter(flag_bert)

In [ ]:
# 1 index values are offensive
# 0 index values are not offensive
# 2 index values are neutral

count = 0
for i in range(0,len(flag_bert)):
      if flag_bert[i] == 1 and labels_bert[i] == 1:
        count = count + 1
print(count)

In [ ]:
for i in range(0,len(flag_bert)):
    if(labels_bert[i] == 0):
      labels_bert[i] = 2
    elif(labels_bert[i] == 1):
      labels_bert[i] = 1
    else:
      labels_bert[i] = 0

In [ ]:
flag_bert = []
count = 0

x_bert = []
y_bert = []
z_bert = []


for i in range(0,len(X)):
    count = count + 1
    x_bert.append(cl_bert[i][0])
    y_bert.append(cl_bert[i][1])
    z_bert.append(cl_bert[i][2])

    if( answer_train[i] > 0.3 and answer_train[i] < 0.7 ):
        flag_bert.append(2)

    if( answer_train[i] > 0 and answer_train[i] <= 0.3 ):
        flag_bert.append(0)

    if( answer_train[i] >= 0.7 and answer_train[i] < 1 ):
        flag_bert.append(1)

print(count)

In [ ]:
con_mat = tf.math.confusion_matrix(labels=flag_bert, predictions=labels_bert)
print(con_mat)

In [ ]:
import sklearn
print(sklearn.metrics.classification_report(flag_bert, labels_bert, output_dict=False, digits=3))

In [ ]:
prob_bert = gmm_bert.predict_proba(cl_bert)
prob_bert = prob_bert.T

In [ ]:
len(y_kmeans_bert)

<h5>GMM PLM Plot</h5>

In [ ]:
pred_colour = []
for i in range(0,len(y_kmeans_bert)):
    if labels_bert[i] == 2:
      pred_colour.append("Neutral")
    if labels_bert[i] == 1:
      pred_colour.append("Not Offensive")
    if labels_bert[i] == 0:
      pred_colour.append("Offensive")

test_df = pd.DataFrame({'SVNS Offensive':prob_bert[2], 'SVNS Non Offensive':prob_bert[1], 'SVNS Neutral':prob_bert[0], 'Labels:':pred_colour})

fig = px.scatter_3d(test_df, x='SVNS Offensive', y='SVNS Non Offensive', z='SVNS Neutral', color='Labels:')
fig.update_traces(
    marker={
        'size': 1.8,
        'opacity': 1,
        'colorscale' : 'viridis',
    }
)
fig.update_layout(legend= {'itemsizing': 'constant', 'font_size':18}, font_size=15, scene_aspectmode='cube')
fig.update_layout(width = 850, height = 750)
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))

<h3>Dense 3 layer</h3>

In [ ]:
from sklearn.preprocessing import normalize

In [ ]:
cl_norm = normalize(cluster_3, norm='l2', axis=1)

In [ ]:
flag_3 = []
count = 0

x_ = []
y_ = []
z_ = []


for i in range(0,len(X)):
    count = count + 1
    x_.append(cl_norm[i][0])
    y_.append(cl_norm[i][1])
    z_.append(cl_norm[i][2])

    if( answer_train[i] > 0.28 and answer_train[i] < 0.8 ):
        flag_3.append(2)

    if( answer_train[i] > 0 and answer_train[i] <= 0.28 ):
        flag_3.append(0)

    if( answer_train[i] >= 0.8 and answer_train[i] < 1 ):
        flag_3.append(1)

print(count)

<p>k-means Dense 3</p>

In [ ]:
kmeans_3 = KMeans(n_clusters=3, random_state=44).fit(cl_norm)
y_kmeans_3 = kmeans_3.predict(cl_norm)

In [ ]:
Counter(y_kmeans_3)

In [ ]:
Counter(flag_3)

In [ ]:
# 1 index values are offensive
# 0 index values are not offensive
# 2 index values are neutral

count = 0
for i in range(0,len(flag_3)):
      if flag_3[i] == 0 and y_kmeans_3[i] == 1:
        count = count + 1
print(count)

In [ ]:
for i in range(0,len(flag_3)):
    if(y_kmeans_3[i] == 0):
      y_kmeans_3[i] = 2
    elif(y_kmeans_3[i] == 1):
      y_kmeans_3[i] = 1
    else:
      y_kmeans_3[i] = 0

In [ ]:
flag_3 = []
count = 0

x_ = []
y_ = []
z_ = []


for i in range(0,len(X)):
    count = count + 1
    x_.append(cl_norm[i][2])
    y_.append(cl_norm[i][1])
    z_.append(cl_norm[i][0])

    if( answer_train[i] > 0.3 and answer_train[i] < 0.7 ):
        flag_3.append(2)

    if( answer_train[i] > 0 and answer_train[i] <= 0.3 ):
        flag_3.append(0)

    if( answer_train[i] >= 0.7 and answer_train[i] < 1 ):
        flag_3.append(1)

print(count)

In [ ]:
Counter(flag_3)

In [ ]:
con_mat = tf.math.confusion_matrix(labels=flag_3, predictions=y_kmeans_3)
print(con_mat)

In [ ]:
import sklearn
print(sklearn.metrics.classification_report(flag_3, y_kmeans_3, output_dict=False, digits=3))

<p>Transition phase</p>

In [ ]:
pred_colour = []
for i in range(0,len(y_kmeans_bert)):
    if y_kmeans_3[i] == 2:
      pred_colour.append("Neutral")
    if y_kmeans_3[i] == 1:
      pred_colour.append("Not Offensive")
    if y_kmeans_3[i] == 0:
      pred_colour.append("Offensive")

test_df = pd.DataFrame({'X':x_, 'Y':y_, 'Z':z_, 'Labels:':pred_colour})

fig = px.scatter_3d(test_df, x='X', y='Y', z='Z', color='Labels:')
fig.update_traces(
    marker={
        'size': 1,
        'opacity': 1,
        'colorscale' : 'viridis',
    }
)
fig.update_layout(legend= {'itemsizing': 'constant'}, font_size=14, scene_aspectmode='cube')
fig.update_layout(width = 850, height = 750)
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))

<p>Original predictions</p>

In [ ]:
pred_colour = []
for i in range(0,len(y_kmeans_bert)):
    if pred_train[i] == 1:
      pred_colour.append("Not Offensive")
    if pred_train[i] == 0:
      pred_colour.append("Offensive")

test_df = pd.DataFrame({'X':x_, 'Y':y_, 'Z':z_, 'Labels:':pred_colour})

fig = px.scatter_3d(test_df, x='X', y='Y', z='Z', color='Labels:')
fig.update_traces(
    marker={
        'size': 1,
        'opacity': 1,
        'colorscale' : 'rainbow',
    }
)
fig.update_layout(legend= {'itemsizing': 'constant'}, font_size=14, scene_aspectmode='cube')
fig.update_layout(width = 850, height = 750)
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))

<h4>End of transition capture</h4>

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cosine

In [ ]:
centers_3 = kmeans_3.cluster_centers_
print(centers_3)

In [ ]:
svns_off = []
for i in range(0,len(Y_test)):
    off = cosine(cl_norm[i], centers_3[2])/2
    svns_off.append(1-off)
print(len(svns_off))

In [ ]:
svns_noff = []
for i in range(0,len(Y_test)):
    noff = cosine(cl_norm[i], centers_3[1])/2
    svns_noff.append(1-noff)
print(len(svns_noff))

In [ ]:
svns_neu = []
for i in range(0,len(Y_test)):
    neu = cosine(cl_norm[i], centers_3[0])/2
    svns_neu.append(1-neu)
print(len(svns_neu))

<h5>k-means Dense 3 Plot</h5>

In [ ]:
pred_colour = []
for i in range(0,len(y_kmeans_bert)):
    if y_kmeans_3[i] == 2:
      pred_colour.append("Neutral")
    if y_kmeans_3[i] == 1:
      pred_colour.append("Not Offensive")
    if y_kmeans_3[i] == 0:
      pred_colour.append("Offensive")

test_df = pd.DataFrame({'SVNS Offensive':svns_off, 'SVNS Not Offensive':svns_noff, 'SVNS Neutral':svns_neu, 'Labels:':pred_colour})

fig = px.scatter_3d(test_df, x='SVNS Offensive', y='SVNS Not Offensive', z='SVNS Neutral', color='Labels:')
fig.update_traces(
    marker={
        'size': 1,
        'opacity': 1,
        'colorscale' : 'viridis',
    }
)
fig.update_layout(legend= {'itemsizing': 'constant'}, font_size=14, scene_aspectmode='cube')
fig.update_layout(width = 850, height = 750)
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))

In [ ]:
pred_droberta = []
for i in range(0,len(Y_test)):
    if(svns_off[i] > svns_noff[i]):
        pred_droberta.append(0)
    else:
        pred_droberta.append(1)
print(classification_report(Y_test, pred_droberta, output_dict=False, digits=3))

In [ ]:
con_mat = tf.math.confusion_matrix(labels=Y_test, predictions=pred_droberta)
print(con_mat)

<p> GMM model </p>

In [ ]:
gmm_3 = GaussianMixture(n_components=3, random_state = 44).fit(cl_norm)

In [ ]:
mean_norm = gmm_3.means_
cov_norm = gmm_3.covariances_
print(np.shape(mean_norm))
print(np.shape(cov_norm))

In [ ]:
labels_norm = gmm_3.predict(cl_norm)

In [ ]:
flag_3 = []
count = 0

x_ = []
y_ = []
z_ = []


for i in range(0,len(X)):
    count = count + 1
    x_.append(cl_norm[i][2])
    y_.append(cl_norm[i][1])
    z_.append(cl_norm[i][0])

    if( answer_train[i] > 0.28 and answer_train[i] < 0.8 ):
        flag_3.append(2)

    if( answer_train[i] > 0 and answer_train[i] <= 0.28 ):
        flag_3.append(0)

    if( answer_train[i] >= 0.8 and answer_train[i] < 1 ):
        flag_3.append(1)

print(count)

In [ ]:
Counter(labels_norm)

In [ ]:
Counter(flag_3)

In [ ]:
# 1 index values are offensive
# 0 index values are not offensive
# 2 index values are neutral

count = 0
for i in range(0,len(flag_3)):
      if flag_3[i] == 0 and labels_norm[i] == 1:
        count = count + 1
print(count)

In [ ]:
for i in range(0,len(flag_3)):
    if(labels_norm[i] == 0):
      labels_norm[i] = 2
    elif(labels_norm[i] == 1):
      labels_norm[i] = 1
    else:
      labels_norm[i] = 0

In [ ]:
flag_3 = []
count = 0

x_ = []
y_ = []
z_ = []


for i in range(0,len(X)):
    count = count + 1
    x_.append(cl_norm[i][2])
    y_.append(cl_norm[i][1])
    z_.append(cl_norm[i][0])

    if( answer_train[i] > 0.3 and answer_train[i] < 0.7 ):
        flag_3.append(2)

    if( answer_train[i] > 0 and answer_train[i] <= 0.3 ):
        flag_3.append(0)

    if( answer_train[i] >= 0.7 and answer_train[i] < 1 ):
        flag_3.append(1)

print(count)

In [ ]:
con_mat = tf.math.confusion_matrix(labels=flag_3, predictions=labels_norm)
print(con_mat)

In [ ]:
import sklearn
print(sklearn.metrics.classification_report(flag_3, labels_norm, output_dict=False, digits=3))

In [ ]:
prob_norm = gmm_3.predict_proba(cl_norm)
prob_norm = prob_norm.T

<h5> GMM Dense 3 Plot</h5>

In [ ]:
pred_colour = []
for i in range(0,len(y_kmeans_bert)):
    if labels_norm[i] == 2:
      pred_colour.append("Neutral")
    if labels_norm[i] == 1:
      pred_colour.append("Not Offensive")
    if labels_norm[i] == 0:
      pred_colour.append("Offensive")

test_df = pd.DataFrame({'SVNS Offensive':prob_norm[2], 'SVNS Not Offensive':prob_norm[1], 'SVNS Neutral':prob_norm[0], 'Labels:':pred_colour})

fig = px.scatter_3d(test_df, x='SVNS Offensive', y='SVNS Not Offensive', z='SVNS Neutral', color='Labels:')
fig.update_traces(
    marker={
        'size': 1.5,
        'opacity': 1,
        'colorscale' : 'viridis',
    }
)
fig.update_layout(legend= {'itemsizing': 'constant'}, font_size=14, scene_aspectmode='cube')
fig.update_layout(width = 850, height = 750)
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))

<h3>Dense 3 layer end</h3>

<h3>Batch Norm layer</h3>

In [ ]:
model.layers[-4].name

In [ ]:
cluster_32 = keras.Model(inputs=model.input, outputs=model.layers[-4].output)
with strategy.scope():
    cl_32 = cluster_32.predict([test_encodings["input_ids"], test_encodings["attention_mask"]])

In [ ]:
flag_32 = []
count = 0

x_32 = []
y_32 = []
z_32 = []


for i in range(0,len(X)):
    count = count + 1
    x_32.append(cl_32[i][0])
    y_32.append(cl_32[i][1])
    z_32.append(cl_32[i][2])

    if( answer_train[i] > 0.28 and answer_train[i] < 0.8 ):
        flag_32.append(2)

    if( answer_train[i] > 0 and answer_train[i] <= 0.28 ):
        flag_32.append(0)

    if( answer_train[i] >= 0.8 and answer_train[i] < 1 ):
        flag_32.append(1)

print(count)

<p>k-means BatchNorm layer</p>

In [ ]:
kmeans_32 = KMeans(n_clusters=3, random_state=44).fit(cl_32)
y_kmeans_32 = kmeans_32.predict(cl_32)

In [ ]:
Counter(y_kmeans_32)

In [ ]:
Counter(flag_32)

In [ ]:
# 2 index values are offensive
# 0 index values are not offensive
# 0 index values are neutral

count = 0
for i in range(0,len(y_kmeans_32)):
      if flag_32[i] == 2 and y_kmeans_32[i] == 1:
        count = count + 1
print(count)

In [ ]:
for i in range(0,len(y_kmeans_32)):
    if(y_kmeans_32[i] == 0):
      y_kmeans_32[i] = 1
    elif(y_kmeans_32[i] == 1):
      y_kmeans_32[i] = 2
    else:
      y_kmeans_32[i] = 0

In [ ]:
flag_32 = []
count = 0

x_32 = []
y_32 = []
z_32 = []


for i in range(0,len(X)):
    count = count + 1
    x_32.append(cl_32[i][0])
    y_32.append(cl_32[i][1])
    z_32.append(cl_32[i][2])

    if( answer_train[i] > 0.3 and answer_train[i] < 0.7 ):
        flag_32.append(2)

    if( answer_train[i] > 0 and answer_train[i] <= 0.3 ):
        flag_32.append(0)

    if( answer_train[i] >= 0.7 and answer_train[i] < 1 ):
        flag_32.append(1)

print(count)

In [ ]:
con_mat = tf.math.confusion_matrix(labels=flag_32, predictions=y_kmeans_32)
print(con_mat)

In [ ]:
import sklearn
print(sklearn.metrics.classification_report(flag_32, y_kmeans_32, output_dict=False, digits=3))

In [ ]:
centers_32 = kmeans_32.cluster_centers_

In [ ]:
svns_off = []
for i in range(0,len(Y_test)):
    off = cosine(cl_32[i], centers_32[1])/2
    svns_off.append(1-off)
print(len(svns_off))

In [ ]:
svns_noff = []
for i in range(0,len(Y_test)):
    noff = cosine(cl_32[i], centers_32[2])/2
    svns_noff.append(1-noff)
print(len(svns_noff))

In [ ]:
svns_neu = []
for i in range(0,len(Y_test)):
    neu = cosine(cl_32[i], centers_32[0])/2
    svns_neu.append(1-neu)
print(len(svns_neu))

<p>k-means BatchNorm Plot</p>

In [ ]:
pred_colour = []
for i in range(0,len(y_kmeans_bert)):
    if y_kmeans_32[i] == 2:
      pred_colour.append("Neutral")
    if y_kmeans_32[i] == 1:
      pred_colour.append("Not Offensive")
    if y_kmeans_32[i] == 0:
      pred_colour.append("Offensive")

test_df = pd.DataFrame({'SVNS Offensive':svns_off, 'SVNS Not Offensive':svns_noff, 'SVNS Neutral':svns_neu, 'Labels:':pred_colour})

fig = px.scatter_3d(test_df, x='SVNS Offensive', y='SVNS Not Offensive', z='SVNS Neutral', color='Labels:')
fig.update_traces(
    marker={
        'size': 1,
        'opacity': 1,
        'colorscale' : 'viridis',
    }
)
fig.update_layout(legend= {'itemsizing': 'constant'}, font_size=14, scene_aspectmode='cube')
fig.update_layout(width = 850, height = 750)
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))

In [ ]:
pred_BNroberta = []
for i in range(0,len(Y_test)):
    if(svns_off[i] > svns_neu[i]):
        pred_BNroberta.append(0)
    else:
        pred_BNroberta.append(1)
print(classification_report(Y_test, pred_BNroberta, output_dict=False, digits=3))

In [ ]:
con_mat = tf.math.confusion_matrix(labels=Y_test, predictions=pred_BNroberta)
print(con_mat)

<p> GMM Model </p>

In [ ]:
gmm_32 = GaussianMixture(n_components=3, random_state = 44).fit(cl_32)

In [ ]:
mean_32 = gmm_32.means_
cov_32 = gmm_32.covariances_
print(np.shape(mean_32))
print(np.shape(cov_32))

In [ ]:
labels_32 = gmm_32.predict(cl_32)

In [ ]:
flag_32 = []
count = 0

x_32 = []
y_32 = []
z_32 = []


for i in range(0,len(X)):
    count = count + 1
    x_32.append(cl_32[i][0])
    y_32.append(cl_32[i][1])
    z_32.append(cl_32[i][2])

    if( answer_train[i] > 0.28 and answer_train[i] < 0.8 ):
        flag_32.append(2)

    if( answer_train[i] > 0 and answer_train[i] <= 0.28 ):
        flag_32.append(0)

    if( answer_train[i] >= 0.8 and answer_train[i] < 1 ):
        flag_32.append(1)

print(count)

In [ ]:
Counter(flag_32)

In [ ]:
# 1 index values are offensive
# 0 index values are not offensive
# 2 index values are neutral

count = 0
for i in range(0,len(flag_32)):
      if flag_32[i] == 1 and labels_32[i] == 1:
        count = count + 1
print(count)

In [ ]:
for i in range(0,len(flag_32)):
    if(labels_32[i] == 0):
      labels_32[i] = 2
    elif(labels_32[i] == 1):
      labels_32[i] = 1
    else:
      labels_32[i] = 0

In [ ]:
flag_32 = []
count = 0

x_32 = []
y_32 = []
z_32 = []


for i in range(0,len(X)):
    count = count + 1
    x_32.append(cl_32[i][0])
    y_32.append(cl_32[i][1])
    z_32.append(cl_32[i][2])

    if( answer_train[i] > 0.3 and answer_train[i] < 0.7 ):
        flag_32.append(2)

    if( answer_train[i] > 0 and answer_train[i] <= 0.3 ):
        flag_32.append(0)

    if( answer_train[i] >= 0.7 and answer_train[i] < 1 ):
        flag_32.append(1)

print(count)

In [ ]:
con_mat = tf.math.confusion_matrix(labels=flag_32, predictions=labels_32)
print(con_mat)

In [ ]:
import sklearn
print(sklearn.metrics.classification_report(flag_32, labels_32, output_dict=False, digits=3))

In [ ]:
prob_32 = gmm_32.predict_proba(cl_32)
prob_32 = prob_32.T

<p>GMM BatchNorm Plot</p>

In [ ]:
pred_colour = []
for i in range(0,len(y_kmeans_bert)):
    if labels_32[i] == 2:
      pred_colour.append("Neutral")
    if labels_32[i] == 1:
      pred_colour.append("Not Offensive")
    if labels_32[i] == 0:
      pred_colour.append("Offensive")

test_df = pd.DataFrame({'SVNS Offensive':prob_32[2], 'SVNS Not Offensive':prob_32[1], 'SVNS Neutral':prob_32[0], 'Labels:':pred_colour})

fig = px.scatter_3d(test_df, x='SVNS Offensive', y='SVNS Not Offensive', z='SVNS Neutral', color='Labels:')
fig.update_traces(
    marker={
        'size': 1.5,
        'opacity': 1,
        'colorscale' : 'viridis',
    }
)
fig.update_layout(legend= {'itemsizing': 'constant'}, font_size=14, scene_aspectmode='cube')
fig.update_layout(width = 850, height = 750)
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))